In [ ]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp results

<IPython.core.display.Javascript object>

# module Results

> Handle benchmark results

In [ ]:
# export

import math

from typing import Optional
from pydantic import BaseModel

from will_it_saturate.epochs import Epoch
from will_it_saturate.hosts import HostDetails
from will_it_saturate.servers import BaseServer
from will_it_saturate.clients import BaseClient

<IPython.core.display.Javascript object>

# Benchmark Result

In [ ]:
# export


def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return s, size_name[i]


class Result(BaseModel):
    server: BaseServer
    client: BaseClient
    server_details: Optional[HostDetails]
    client_details: Optional[HostDetails]
    file_size: int
    elapsed: Optional[float] = None
    complete_size: int

    def __hash__(self):
        return hash(self.json(exclude={"elapsed"}))

    def __eq__(self, other):
        self_dict, other_dict = self.dict(exclude={"elapsed"}), other.dict(
            exclude={"elapsed"}
        )
        return self_dict == other_dict

    @classmethod
    def build_empty_result(cls, row, server, client):
        return cls(
            server=server,
            client=client,
            file_size=row.file_size,
            complete_size=row.complete_size,
        )

    def make_readable(self, size_in_bytes):
        size, unit = convert_size(size_in_bytes)
        return f"{size}{unit}"

    @property
    def readable_file_size(self):
        return self.make_readable(self.file_size)

    @property
    def bytes_per_second(self):
        return self.complete_size / self.elapsed

    @property
    def readable_bytes_per_second(self):
        return self.make_readable(self.bytes_per_second)

    def dict_with_properties(self):
        return {
            **super().dict(),
            "file_size_h": self.readable_file_size,
            "bytes_per_second": self.bytes_per_second,
            "bytes_per_second_h": self.readable_bytes_per_second,
        }

    def dict_for_pandas(self):
        return {
            "server": self.server.name,
            "client": self.client.name,
            "server_host": self.server_details.machine_id,
            "client_host": self.client_details.machine_id,
            "elapsed": self.elapsed,
            "file_size": self.file_size,
            "file_size_h": self.readable_file_size,
            "complete_size": self.complete_size,
            "bytes_per_second": self.bytes_per_second,
            "bytes_per_second_h": self.readable_bytes_per_second,
        }

<IPython.core.display.Javascript object>

## Usage

In [ ]:
file_size = 10 ** 6
complete_size = 100 * file_size
dummy_details = HostDetails(machine_id="asdf", cpuinfo={"arch": "X86_64"})
result = Result(
    server_details=dummy_details,
    client_details=dummy_details,
    server=BaseServer(name="nginx"),
    client=BaseClient(name="httpx"),
    file_size=file_size,
    elapsed=3.0,
    complete_size=complete_size,
)
print(result.dict())
print(result.dict_with_properties())
print(result.dict_for_pandas())

{'server': {'protocol': 'http', 'name': 'nginx', 'host': 'localhost', 'port': 8000}, 'client': {'name': 'httpx'}, 'server_details': {'machine_id': 'asdf', 'cpuinfo': {'arch': 'X86_64'}}, 'client_details': {'machine_id': 'asdf', 'cpuinfo': {'arch': 'X86_64'}}, 'file_size': 1000000, 'elapsed': 3.0, 'complete_size': 100000000}
{'server': {'protocol': 'http', 'name': 'nginx', 'host': 'localhost', 'port': 8000}, 'client': {'name': 'httpx'}, 'server_details': {'machine_id': 'asdf', 'cpuinfo': {'arch': 'X86_64'}}, 'client_details': {'machine_id': 'asdf', 'cpuinfo': {'arch': 'X86_64'}}, 'file_size': 1000000, 'elapsed': 3.0, 'complete_size': 100000000, 'file_size_h': '976.56KB', 'bytes_per_second': 33333333.333333332, 'bytes_per_second_h': '31.79MB'}
{'server': 'nginx', 'client': 'httpx', 'server_host': 'asdf', 'client_host': 'asdf', 'elapsed': 3.0, 'file_size': 1000000, 'file_size_h': '976.56KB', 'complete_size': 100000000, 'bytes_per_second': 33333333.333333332, 'bytes_per_second_h': '31.79MB

<IPython.core.display.Javascript object>

In [ ]:
result.server_details.machine_id

'asdf'

<IPython.core.display.Javascript object>

## Tests

In [ ]:
assert result.readable_bytes_per_second == "31.79MB"
assert "file_size" in result.json()

<IPython.core.display.Javascript object>

In [ ]:
from will_it_saturate.clients import BaseClient
from will_it_saturate.servers import BaseServer


class Client(BaseClient):
    measured: bool = False

    def measure(self, benchmark_row):
        self.measured = True
        print("measure_benchmark_row: ", benchmark_row)
        return 2.0


class Server(BaseServer):
    started: bool = False
    stopped: bool = False

    def start(self):
        self.started = True

    def stop(self):
        self.stopped = True


row_params = {
    "file_size": 10 ** 6 * 10,
    "duration": 30,
    "bandwidth": 10 ** 9 / 8,
    "file_creator_name": "dummy",
}
epoch = Epoch(**row_params)


client = Client(name="client")
server = Server(name="server")

result = Result.build_empty_result(epoch, server, client)

<IPython.core.display.Javascript object>

In [ ]:
assert result.server == server
assert result.client == client

<IPython.core.display.Javascript object>

# Export

In [ ]:
# dont_test

from nbdev.export import notebook2script

notebook2script()

Converted 00_host.ipynb.
Converted 01_django_views.ipynb.
Converted 01_fastapi_views.ipynb.
Converted 01_file.ipynb.
Converted 05_epochs.ipynb.
Converted 06_results.ipynb.
Converted 25_core.ipynb.
Converted 27_control_server.ipynb.
Converted 28_control_client.ipynb.
Converted 29_control_cli.ipynb.
Converted 30_servers.ipynb.
Converted 31_local_servers.ipynb.
Converted 32_docker_servers.ipynb.
Converted 40_clients.ipynb.
Converted 41_benchmark_clients.ipynb.
Converted 70_repositories.ipynb.
Converted 71_sqlite_repository.ipynb.
Converted 90_benchmark_without_benchmark.ipynb.
Converted 98_run_benchmark.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>